Using imgaug to generate 35 augmented images as well as their corresponding bounding box annotations from the "sample garbage space" image and its corresponding BB annotations

In [1]:
pip install imgaug opencv-python

Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at c:\users\snepo\appdata\local\programs\python\python311\lib\site-packages\hyperopt-0.2.7-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\snepo\appdata\local\programs\python\python311\lib\site-packages\numpydoc-1.6.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\snepo\appdata\local\programs\python\python311\lib\site-packages\pathlib-1.0.1-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\snepo\appd

In [2]:
import imgaug.augmenters as iaa
import imgaug as ia
import numpy as np
import cv2
import json
import os

# Sample annotation data
annotation_data = {
    "boxes": [
        {"label": "garbage", "x": 97.73, "y": 70.72, "width": 20.26, "height": 20.24},
        {"label": "garbage", "x": 161.57, "y": 135.38, "width": 18.95, "height": 20.56},
        {"label": "garbage", "x": 259.07, "y": 167.03, "width": 23.93, "height": 22.87},
        {"label": "garbage", "x": 291.81, "y": 199.24, "width": 22.43, "height": 23.28}
    ],
    "height": 240,
    "key": "sample garbage space.jpg",
    "width": 320
}

# Load the image
image = cv2.imread(annotation_data['key'])
height, width = annotation_data['height'], annotation_data['width']

# Convert bounding boxes to imgaug format
bbs = []
for box in annotation_data['boxes']:
    x1 = box['x']
    y1 = box['y']
    x2 = x1 + box['width']
    y2 = y1 + box['height']
    bbs.append(ia.BoundingBox(x1=x1, y1=y1, x2=x2, y2=y2))

bbs = ia.BoundingBoxesOnImage(bbs, shape=image.shape)

# Define augmentation sequence
seq = iaa.Sequential([
    iaa.Fliplr(0.5),  # horizontal flips
    iaa.Crop(percent=(0, 0.1)),  # random crops
    iaa.LinearContrast((0.75, 1.5)),  # improve or worsen the contrast
    iaa.AdditiveGaussianNoise(scale=(0, 0.05*255)),  # add gaussian noise
    iaa.Multiply((0.8, 1.2)),  # change brightness
    iaa.Affine(
        scale={"x": (0.8, 1.2), "y": (0.8, 1.2)},  # scale images
        translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)},  # translate images
        rotate=(-25, 25),  # rotate images
        shear=(-8, 8)  # shear images
    )
])

# Create output directory if it doesn't exist
output_dir = "image/output4"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Augment images and save with annotations
for i in range(35):
    # Augment image and bounding boxes
    image_aug, bbs_aug = seq(image=image, bounding_boxes=bbs)

    # Clip bounding boxes to image size
    bbs_aug = bbs_aug.remove_out_of_image().clip_out_of_image()

    # Save augmented image
    augmented_image_filename = os.path.join(output_dir, f"augmented_image_{i+1}.jpg")
    cv2.imwrite(augmented_image_filename, image_aug)

    # Convert augmented bounding boxes back to the required format
    augmented_boxes = []
    for bb in bbs_aug.bounding_boxes:
        augmented_boxes.append({
            "label": "garbage",
            "x": bb.x1,
            "y": bb.y1,
            "width": bb.width,
            "height": bb.height
        })

    # Save augmented annotations
    augmented_annotation = {
        "boxes": augmented_boxes,
        "height": height,
        "key": augmented_image_filename,
        "width": width
    }
    
    annotation_filename = os.path.join(output_dir, f"augmented_annotation_{i+1}.json")
    with open(annotation_filename, "w") as f:
        json.dump(augmented_annotation, f)

print(f"Generated 35 augmented images and annotations in '{output_dir}' directory.")


Generated 35 augmented images and annotations in 'image/output4' directory.
